In [5]:
def peel_first_col(fname, fcol, delimiter=None):
    with open(fname, 'r') as fin:
        for line in fin:
            try:
                fcol.append(line.split(delimiter, 1)[0])
                yield line.split(delimiter, 1)[1]
            except IndexError:
                continue

In [6]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
import networkx as nx
import string

symMats = []
graphs = []
algos = []
col = []
algoOrder = ["PoAV", "PQAV", "Po2P", "PQ2P", "PoPN", "PQPN"]
ext =       ["",     "_1",   "_2",   "_3",   "_4",   "_5"]
for e in range(len(ext)):
    for f in range(5):
        col = []
        klDiv = np.loadtxt(peel_first_col("../output/testCMs/r"+str(f+1)+ext[e]+".matrix", col), skiprows=1)
        col.pop(0)
        #klDiv = np.absolute(klDiv)
        symKlDiv = np.triu(klDiv)+np.tril(klDiv).T
        symMats = symMats + [symKlDiv]
        
        G = nx.from_numpy_matrix(symKlDiv)
        for i in range(len(col)):
            G.node[i]['id'] = col[i]
        graphs = graphs + [G]
        algos = algos + [algoOrder[e] + str(f)]
print(np.array(col).T)

['m_753.fasta.motif.h1_3.h2_3.h1_1.h1_4.h1_2_chr5.sto.cm'
 'm_9.fasta.motif.h1_4.h2_1.h1_3_chr5.sto.cm'
 'm_1540.fasta.motif.h1_2.h1_4.h1_3_chr5.sto.cm'
 'm_1539.fasta.motif.h2_2.h2_3_chr5.sto.cm'
 'm_1720.fasta.motif.h2_2.h2_5_chr5.sto.cm'
 'm_391.fasta.motif.h2_3.h2_4_chr5.sto.cm'
 'm_417.fasta.motif.h1_5_chr5.sto.cm']


In [7]:
import numpy as np
from sklearn.cluster import SpectralClustering
from sklearn import metrics

for i in range(len(symMats)):
    adj_mat = symMats[i] + symMats[i].T
    aff_mat = np.exp(-1 * adj_mat / adj_mat.std())

    sc = SpectralClustering(3, affinity='precomputed', n_init=100)
    sc.fit(aff_mat)
    for n in range(len(sc.labels_)):
        graphs[i].node[n]['group'] = str(sc.labels_[n])
    
    # print(str(algos[i]), "\t", sc.labels_)
    print(str(algos[i]), "\n", symMats[i])
    if (i % 5 == 4):
        print()

PoAV0 
 [[  0.     116.451  127.1559 122.7222 135.3996 139.9891 112.9817]
 [  0.       0.     110.1554 115.6374 130.0208 135.8775  99.9079]
 [  0.       0.       0.      12.0407 130.8657 138.1505 103.7424]
 [  0.       0.       0.       0.     129.3254 137.6009 104.5831]
 [  0.       0.       0.       0.       0.     148.3967 118.2296]
 [  0.       0.       0.       0.       0.       0.     117.2208]
 [  0.       0.       0.       0.       0.       0.       0.    ]]
PoAV1 
 [[  0.     121.8972 121.675  124.3437 138.4912 140.8083 110.1548]
 [  0.       0.     110.989  117.2397 130.1915 131.947   97.488 ]
 [  0.       0.       0.      14.1702 130.4051 138.862  103.2086]
 [  0.       0.       0.       0.     131.4786 137.5343 106.1001]
 [  0.       0.       0.       0.       0.     152.3268 117.5728]
 [  0.       0.       0.       0.       0.       0.     117.626 ]
 [  0.       0.       0.       0.       0.       0.       0.    ]]
PoAV2 
 [[  0.     121.6201 122.9585 126.2866 139.8361 136

PQ2P1 
 [[  0.         202.89707122 188.31413854 201.74198733 221.79938201
  223.15034255 174.0400477 ]
 [  0.           0.         189.83852346 202.68041962 192.97949134
  211.04623312 152.3728766 ]
 [  0.           0.           0.           8.74479372 191.85207673
  217.50045663 159.14783648]
 [  0.           0.           0.           0.         223.09283587
  225.121442   164.9553062 ]
 [  0.           0.           0.           0.           0.
  241.74965345 171.1572994 ]
 [  0.           0.           0.           0.           0.
    0.         183.72934708]
 [  0.           0.           0.           0.           0.
    0.           0.        ]]
PQ2P2 
 [[  0.         192.92353559 217.96314203 225.12098553 212.36913697
  210.04925056 174.73767253]
 [  0.           0.         193.68881837 197.16876893 197.27882151
  227.22179213 153.17553619]
 [  0.           0.           0.          -3.00985277 211.01893622
  223.5882624  158.65405525]
 [  0.           0.           0.           0.  

PQPN4 
 [[  0.         141.65699637 153.77511727 178.82870029 196.70683568
  188.15499008 102.31914748]
 [  0.           0.          78.13831636 143.37409897 161.9553894
  158.83390043 117.77215969]
 [  0.           0.           0.           8.49623544 176.30932207
  158.00670491 118.06211484]
 [  0.           0.           0.           0.         153.49534724
  155.78305001 178.50179619]
 [  0.           0.           0.           0.           0.
  230.81858175 179.3287775 ]
 [  0.           0.           0.           0.           0.
    0.         149.46916437]
 [  0.           0.           0.           0.           0.
    0.           0.        ]]



In [8]:
for i in range(len(graphs)):
    G=graphs[i]
    nx.write_gexf(G,"../output/"+str(algos[i])+".gexf")